In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cf
import matplotlib.dates as mdates  #Importing module for date formatting
import matplotlib.ticker as ticker  # Importing ticker for customizing colorbar ticks
from scipy.ndimage import gaussian_filter
import seaborn as sns
import glob
from datetime import datetime

In [2]:
# path = '/glade/work/clackner/output_crsim_les/'
# data = 'crsim_output_*'
# ds = sorted(glob.glob(f"{path}{data}"))  # total size for d02 and d04 are 25913
# # ds = np.sort(glob.glob(f"{path}{data}"))
# # select d02 files
# d02_files = [file for file in ds if 'd02' in file]  # d02 files 18579....dx=150m

# # select d04 files
# # d04_files = [file for file in ds if 'd04' in file] # d02 files 7334....dx=30m

# path = '/glade/work/clackner/output_crsim_les/'
# data = 'crsim_output_les.d02.*'
# ds = sorted(glob.glob(f"{path}{data}")) # total size = 18534 without the snow, rain, ice, graupel file
# # If 10 files represent 1 minute, with each file corresponding to a 6-second interval, then 18534 files would represent approximately 1853.4 minutes.
# # 1853.4 minutes is equivalent to approximately 30.89 hours.

In [24]:
path = '/glade/work/clackner/output_crsim_les/'
data = 'crsim_output_les.d02.*'
file_paths = sorted(glob.glob(f"{path}{data}"))
len(file_paths)

18534

In [25]:
file_paths[:10]

['/glade/work/clackner/output_crsim_les/crsim_output_les.d02.000000',
 '/glade/work/clackner/output_crsim_les/crsim_output_les.d02.000006',
 '/glade/work/clackner/output_crsim_les/crsim_output_les.d02.000012',
 '/glade/work/clackner/output_crsim_les/crsim_output_les.d02.000018',
 '/glade/work/clackner/output_crsim_les/crsim_output_les.d02.000024',
 '/glade/work/clackner/output_crsim_les/crsim_output_les.d02.000030',
 '/glade/work/clackner/output_crsim_les/crsim_output_les.d02.000036',
 '/glade/work/clackner/output_crsim_les/crsim_output_les.d02.000042',
 '/glade/work/clackner/output_crsim_les/crsim_output_les.d02.000048',
 '/glade/work/clackner/output_crsim_les/crsim_output_les.d02.000054']

In [34]:
time_dim = pd.date_range(start="2020-03-13", periods=len(file_paths), freq='4.6S')
time_dim

DatetimeIndex([       '2020-03-13 00:00:00', '2020-03-13 00:00:04.600000',
               '2020-03-13 00:00:09.200000', '2020-03-13 00:00:13.800000',
               '2020-03-13 00:00:18.400000',        '2020-03-13 00:00:23',
               '2020-03-13 00:00:27.600000', '2020-03-13 00:00:32.200000',
               '2020-03-13 00:00:36.800000', '2020-03-13 00:00:41.400000',
               ...
               '2020-03-13 23:40:10.400000',        '2020-03-13 23:40:15',
               '2020-03-13 23:40:19.600000', '2020-03-13 23:40:24.200000',
               '2020-03-13 23:40:28.800000', '2020-03-13 23:40:33.400000',
                      '2020-03-13 23:40:38', '2020-03-13 23:40:42.600000',
               '2020-03-13 23:40:47.200000', '2020-03-13 23:40:51.800000'],
              dtype='datetime64[ns]', length=18534, freq='4600L')

In [28]:
%%time
datasets = [xr.open_dataset(i) for i in file_paths[::]]
# Generate the time dimension
time_dim = pd.date_range(start="2020-03-13", periods=len(file_paths), freq='4.5S')

# Assuming each dataset corresponds to a single time point,
# add the time dimension to each dataset
for i, ds in enumerate(datasets):
    # Add the time dimension as a coordinate
    ds = ds.assign_coords(time=time_dim[i])

    # If you need the time as a dimension in each variable,
    # you might need to expand dimensions
    ds = ds.expand_dims("time")

    # Update the dataset in the list
    datasets[i] = ds

print("Done adding time")
print("==================")
print("Concatenating all files")
# Concatenate the datasets along the time dimension
combined_dataset = xr.concat(datasets, dim='time')
print("DONE!")

# Now, each dataset in 'datasets' has a 'time' coordinate
# Save the combined dataset to a new netCDF file
combined_dataset.to_netcdf("cr-sim-output_file-4.5sec.nc")
print("done saving files!!!")
print("DONE!")

Done adding time
Concatenating all files
DONE!
done saving files!!!
DONE!
CPU times: user 15min 49s, sys: 27.4 s, total: 16min 17s
Wall time: 22min 55s


In [30]:
xr.open_dataset('./cr-sim-output_file-4.5sec.nc')

<xarray.Dataset>
Dimensions:                        (time: 18534, ny: 1, nx: 1, nz: 135, one: 1,
                                    n_layers: 10)
Coordinates:
  * time                           (time) datetime64[ns] 2020-03-13 ... 2020-...
Dimensions without coordinates: ny, nx, nz, one, n_layers
Data variables: (12/50)
    xlat                           (time, ny, nx) float32 ...
    xlong                          (time, ny, nx) float32 ...
    Zhh                            (time, nz, ny, nx) float32 ...
    DV                             (time, nz, ny, nx) float32 ...
    SWh                            (time, nz, ny, nx) float32 ...
    SWt                            (time, nz, ny, nx) float32 ...
    ...                             ...
    azim                           (time, nz, ny, nx) float32 ...
    range                          (time, nz, ny, nx) float32 ...
    height                         (time, nz, ny, nx) float32 ...
    wcont                          (time, nz, ny, nx) float32 ...
    x_scene                        (time, nx) float32 ...
    y_scene                        (time, ny) float32 ...
Attributes: (12/15)
    description:                        forward radar simulator output
    model_version:                      crsim_v3.3
    WRF_input_file:                     profiles.d02.000000.nc
    MP_PHYSICS:                             8
    x_indices_of_WRF_extracted_scene:   1 - 1
    y_indices_of_WRF_extracted_scene:   1 - 1
    ...                                 ...
    x_and_y_indices_of_radar_position:  1 , 1
    height_of_radar:                    10.0 m
    scanning_mode:                      elevation of each scene pixel is rela...
    created_by:                         Aleksandra Tatarevic
    institute:                          http://wwww.clouds.mcgill.ca
    websites:                           https://www.bnl.gov/CMAS/cr-sim.php; ...

In [35]:
pwd

'/glade/work/noteng/masters-research/comble/LES'

In [ ]:
# these are not part
# ['/glade/work/clackner/output_crsim_les/crsim_output_les.d02_snow.000000',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_snow.000006',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_snow.000012',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_snow.072030',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_snow.072024',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_snow.072018',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_snow.072012',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_snow.072006',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_snow.072000',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_rain.072048',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_rain.072042',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_rain.072036',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_rain.072030',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_rain.072024',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_rain.072018',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_rain.072012',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_rain.072006',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_rain.072000',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_ice.072048',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_ice.072042',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_ice.072036',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_ice.072030',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_ice.072024',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_ice.072018',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_ice.072012',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_ice.072006',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_ice.072000',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_graupel.072048',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_graupel.072042',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_graupel.072036',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_graupel.072030',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_graupel.072024',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_graupel.072018',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_graupel.072012',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_graupel.072006',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_graupel.072000',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_cloud.072048',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_cloud.072042',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_cloud.072036',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_cloud.072030',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_cloud.072024',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_cloud.072018',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_cloud.072012',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_cloud.072006',
#  '/glade/work/clackner/output_crsim_les/crsim_output_les.d02_cloud.072000', ]